# Training Doc2Vec on Wikipedia articles

This notebook replicates the **Document Embedding with Paragraph Vectors** paper, http://arxiv.org/abs/1507.07998.

In that paper, the authors only showed results from the DBOW ("distributed bag of words") mode, trained on the English Wikipedia. Here we replicate this experiment using not only DBOW, but also the DM ("distributed memory") mode of the Paragraph Vector algorithm aka Doc2Vec.

## Basic setup

Let's import the necessary modules and set up logging. The code below assumes Python 3.7+ and Gensim 4.0+.

In [1]:
import logging
import multiprocessing
from pprint import pprint

import smart_open
from gensim.corpora.wikicorpus import WikiCorpus, tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Preparing the corpus

First, download the dump of all Wikipedia articles from [here](http://download.wikimedia.org/enwiki/). You want the file named `enwiki-latest-pages-articles.xml.bz2`.

Second, convert that Wikipedia article dump from the arcane Wikimedia XML format into a plain text file. This will make the subsequent training faster and also allow easy inspection of the data = "input eyeballing".

We'll preprocess each article at the same time, normalizing its text to lowercase, splitting into tokens, etc. Below I use a regexp tokenizer that simply looks for alphabetic sequences as tokens. But feel free to adapt the text preprocessing to your own domain. High quality preprocessing is often critical for the final pipeline accuracy – garbage in, garbage out!

In [2]:
wiki = WikiCorpus(
    "enwiki-latest-pages-articles.xml.bz2",  # path to the file you downloaded above
    tokenizer_func=tokenize,  # simple regexp; plug in your own tokenizer here
    metadata=True,  # also return the article titles and ids when parsing
    dictionary={},  # don't start processing the data yet
)

with smart_open.open("wiki.txt.gz", "w", encoding='utf8') as fout:
    for article_no, (content, (page_id, title)) in enumerate(wiki.get_texts()):
        title = ' '.join(title.split())
        if article_no % 500000 == 0:
            logging.info("processing article #%i: %r (%i tokens)", article_no, title, len(content))
        fout.write(f"{title}\t{' '.join(content)}\n")  # title_of_article [TAB] words of the article

/Volumes/work/workspace/gensim/trunk/gensim/utils.py:1332: UserWarning: detected OSX with python3.8+; aliasing chunkize to chunkize_serial
  warnings.warn("detected %s; aliasing chunkize to chunkize_serial" % entity)
2022-03-17 21:15:32,118 : INFO : processing article #0: 'Anarchism' (6538 tokens)
2022-03-17 21:30:00,138 : INFO : processing article #500000: 'Spiritual Formation Bible' (54 tokens)
2022-03-17 21:40:22,219 : INFO : processing article #1000000: 'Adolf von Liebenberg' (52 tokens)
2022-03-17 21:49:43,825 : INFO : processing article #1500000: 'Small nucleolar RNA U6-53/MBII-28' (123 tokens)
2022-03-17 21:59:23,620 : INFO : processing article #2000000: 'Xie Fei' (50 tokens)
2022-03-17 22:09:17,460 : INFO : processing article #2500000: 'Rhein, Saskatchewan' (185 tokens)
2022-03-17 22:19:39,293 : INFO : processing article #3000000: 'Kunyinsky District' (969 tokens)
2022-03-17 22:30:41,221 : INFO : processing article #3500000: 'Lake Saint-Charles' (555 tokens)
2022-03-17 22:41:17

The above takes about 2 hours on my 2021 M1 MacbookPro, and creates a new ~5.8 GB file named `wiki.txt.gz`. We're compressing the text into `.gz` (GZIP) right away to save on disk space, using the [smart_open](https://github.com/RaRe-Technologies/smart_open) library.

Next we'll set up a stream to load the preprocessed articles from `wiki.txt.gz` one by one, in the format expected by Doc2Vec, ready for training. We don't want to load everything into RAM at once, because that would blow up the memory. And it is not necessary – Gensim can handle streamed training data:

In [3]:
class TaggedWikiCorpus:
    def __init__(self, wiki_text_path):
        self.wiki_text_path = wiki_text_path
        
    def __iter__(self):
        for line in smart_open.open(self.wiki_text_path, encoding='utf8'):
            title, words = line.split('\t')
            yield TaggedDocument(words=words.split(), tags=[title])

documents = TaggedWikiCorpus('wiki.txt.gz')  # A streamed iterable; nothing in RAM yet.

In [4]:
# Load and print the first preprocessed Wikipedia document, as a sanity check = "input eyeballing".
first_doc = next(iter(documents))
print(first_doc.tags, ': ', ' '.join(first_doc.words[:50] + ['………'] + first_doc.words[-50:]))

['Anarchism'] :  anarchism is political philosophy and movement that is sceptical of authority and rejects all involuntary coercive forms of hierarchy anarchism calls for the abolition of the state which it holds to be unnecessary undesirable and harmful as historically left wing movement placed on the farthest left of the political spectrum ……… criticism of philosophical anarchism defence of philosophical anarchism stating that both kinds of anarchism philosophical and political anarchism are philosophical and political claims anarchistic popular fiction novel an argument for philosophical anarchism external links anarchy archives anarchy archives is an online research center on the history and theory of anarchism


The document seems legit so let's move on to finally training some Doc2vec models.

## Training Doc2Vec

The original paper had a vocabulary size of 915,715 word types, so we'll try to match it by setting `max_final_vocab` to 1,000,000 in the Doc2vec constructor.

Other critical parameters were left unspecified in the paper, so we'll go with a default window size of five (a prediction window of 5 tokens to either side), and downsampling of frequent words at 1e-5. It looks like the authors tried vector dimensionality of 100, 300, 1,000 & 10,000 in the paper (with 10k dims performing the best), but I'll only train with 300 dimensions here, to keep RAM in check on my laptop.

Feel free to tinker with these values yourself if you like:

In [5]:
workers = multiprocessing.cpu_count()  # train with 10 threads on my 10-core laptop

models = [
    # PV-DBOW: paragraph vector in distributed bag of words mode
    Doc2Vec(
        dm=0, dbow_words=1,  # dbow_words=1 to train word vectors at the same time too, not only DBOW
        vector_size=300, window=5, sample=1e-5, epochs=10, workers=workers, max_final_vocab=1000000,
    ),
    # PV-DM: paragraph vector in distributed memory mode
    Doc2Vec(
        dm=1, dm_concat=0, dm_mean=1,  # use average of context word vectors to train DM
        vector_size=300, window=5, sample=1e-5, epochs=10, workers=workers, max_final_vocab=1000000,
    )
]

2022-03-19 19:45:13,743 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dbow+w,d300,n5,w5,mc5,s1e-05,t9>', 'datetime': '2022-03-19T19:45:13.743356', 'gensim': '4.1.3.dev0', 'python': '3.8.9 (default, Oct 26 2021, 07:25:53) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-12.2.1-arm64-arm-64bit', 'event': 'created'}
2022-03-19 19:45:13,745 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d300,n5,w5,mc5,s1e-05,t9>', 'datetime': '2022-03-19T19:45:13.745470', 'gensim': '4.1.3.dev0', 'python': '3.8.9 (default, Oct 26 2021, 07:25:53) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-12.2.1-arm64-arm-64bit', 'event': 'created'}


In [6]:
models[0].build_vocab(documents, progress_per=500000)
print(models[0])

# Save some time by copying the vocabulary structures from the DBOW model to the DM model.
# Both models are built on top of exactly the same data, so there's no need to repeat the vocab-building step.
models[1].reset_from(models[0])
print(models[1])

2022-03-19 19:45:22,230 : INFO : collecting all words and their counts
2022-03-19 19:45:22,237 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2022-03-19 19:47:16,714 : INFO : PROGRESS: at example #500000, processed 654950164 words (5721331 words/s), 3222179 word types, 500000 tags
2022-03-19 19:48:22,015 : INFO : PROGRESS: at example #1000000, processed 1018611068 words (5569047 words/s), 4480366 word types, 1000000 tags
2022-03-19 19:49:14,916 : INFO : PROGRESS: at example #1500000, processed 1305140647 words (5416499 words/s), 5420104 word types, 1500000 tags
2022-03-19 19:50:00,348 : INFO : PROGRESS: at example #2000000, processed 1550245240 words (5395000 words/s), 6188355 word types, 2000000 tags
2022-03-19 19:50:45,274 : INFO : PROGRESS: at example #2500000, processed 1790661139 words (5351430 words/s), 6941128 word types, 2500000 tags
2022-03-19 19:51:30,706 : INFO : PROGRESS: at example #3000000, processed 2028261627 words (5229901 words/s

Doc2Vec<dbow+w,d300,n5,w5,mc5,s1e-05,t9>
Doc2Vec<dm/m,d300,n5,w5,mc5,s1e-05,t9>


Now we’re ready to train Doc2Vec on the entirety of the English Wikipedia. **Warning!** Training this DBOW model takes ~16 hours, and DM ~4 hours, on my 2021 laptop.

In [7]:
for model in models:
    model.train(documents, total_examples=model.corpus_count, epochs=model.epochs, report_delay=30*60)

2022-03-19 19:55:38,660 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 9 workers on 991887 vocabulary and 300 features, using sg=1 hs=0 sample=1e-05 negative=5 window=5 shrink_windows=True', 'datetime': '2022-03-19T19:55:38.660218', 'gensim': '4.1.3.dev0', 'python': '3.8.9 (default, Oct 26 2021, 07:25:53) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-12.2.1-arm64-arm-64bit', 'event': 'train'}
2022-03-19 19:55:39,851 : INFO : EPOCH 1 - PROGRESS: at 0.00% examples, 5854 words/s, in_qsize 0, out_qsize 0
2022-03-19 20:25:39,805 : INFO : EPOCH 1 - PROGRESS: at 48.25% examples, 393044 words/s, in_qsize 18, out_qsize 0
2022-03-19 20:48:10,761 : INFO : EPOCH - 1 : training on 2996051328 raw words (1198675030 effective words) took 3152.1s, 380279 effective words/s
2022-03-19 20:48:11,793 : INFO : EPOCH 2 - PROGRESS: at 0.00% examples, 367086 words/s, in_qsize 16, out_qsize 0
2022-03-19 21:18:11,834 : INFO : EPOCH 2 - PROGRESS: at 46.01% examples, 380787 words/s, in_q

## Similarity interface

After that, let's test both models! The DBOW model shows similar results as the original paper.

First, calculate the most similar Wikipedia articles to the "Machine learning" article. The calculated word vectors and document vectors are stored separately, in `model.wv` and `model.dv` respectively:

In [8]:
for model in models:
    print(model)
    pprint(model.dv.most_similar(positive=["Machine learning"], topn=20))

Doc2Vec<dbow+w,d300,n5,w5,mc5,s1e-05,t9>
[('Deep learning', 0.6759864687919617),
 ('Pattern recognition', 0.6742060780525208),
 ('Supervised learning', 0.6707901358604431),
 ('Artificial neural network', 0.6665838956832886),
 ('Semi-supervised learning', 0.654625654220581),
 ('Outline of machine learning', 0.6472466588020325),
 ('Multi-task learning', 0.646246075630188),
 ('Boosting (machine learning)', 0.641443133354187),
 ('Neural network', 0.637937605381012),
 ('Types of artificial neural networks', 0.6377928853034973),
 ('Incremental learning', 0.637475848197937),
 ('Perceptron', 0.6358065605163574),
 ('Rule induction', 0.6326183676719666),
 ('Early stopping', 0.6307427883148193),
 ('Multilayer perceptron', 0.6273738741874695),
 ('Connectionist expert system', 0.6203325390815735),
 ('Neural Designer', 0.6193098425865173),
 ('Meta learning (computer science)', 0.6182209253311157),
 ('Feature (machine learning)', 0.6175855398178101),
 ('Domain adaptation', 0.6154839396476746)]
Doc2Ve

Both results seem similar and match the results from the paper's Table 1, although not exactly. This is because we don't kno wthe exact parameters and also Wikipedia. FIXME


, but note the DM model took 4x less time train (training 4x faster).

Second, let's calculate the most similar Wikipedia entries to "Lady Gaga" using Paragraph Vector:

In [9]:
for model in models:
    print(model)
    pprint(model.dv.most_similar(positive=["Lady Gaga"], topn=10))

Doc2Vec<dbow+w,d300,n5,w5,mc5,s1e-05,t9>
[('Katy Perry', 0.6756664514541626),
 ('Ariana Grande', 0.6528252363204956),
 ('Miley Cyrus', 0.6260033845901489),
 ('Taylor Swift', 0.6144734621047974),
 ('Britney Spears', 0.6105279922485352),
 ('List of awards and nominations received by Lady Gaga', 0.6099144220352173),
 ('Madonna', 0.5910527110099792),
 ('Christina Aguilera', 0.5906776785850525),
 ('Beyoncé', 0.5887077450752258),
 ('Demi Lovato', 0.5824941992759705)]
Doc2Vec<dm/m,d300,n5,w5,mc5,s1e-05,t9>
[('Taylor Swift', 0.5584290623664856),
 ('Joanne (album)', 0.5531442761421204),
 ('List of Lady Gaga live performances', 0.546563982963562),
 ('Katy Perry', 0.5364790558815002),
 ('Cynthia Germanotta', 0.5363910794258118),
 ('Artpop', 0.5359800457954407),
 ('Natali Germanotta', 0.5320610404014587),
 ('Beautiful, Dirty, Rich', 0.5298762917518616),
 ('Joanne Trattoria Cookbook', 0.5219755172729492),
 ('The Fame', 0.5171768069267273)]


The DBOW results are in line with what the paper shows in Table 2a), revealing similar singers in the U.S.

Finally, let's do some of the wilder arithmetics that vectors embeddings are famous for. What are the entries most similar to "Lady Gaga" - "American" + "Japanese"? Table 2b) in the paper.

Note that "American" and "Japanese" are word vectors, but they live in the same space as the document vectors so we can add / subtract them at will, for some interesting results. All word vectors were already lowercased by our tokenizer above, so we look for the lowercased version here:

In [10]:
for model in models:
    print(model)
    vec = [model.dv["Lady Gaga"] - model.wv["american"] + model.wv["japanese"]]
    pprint([m for m in model.dv.most_similar(vec, topn=11) if m[0] != "Lady Gaga"])

Doc2Vec<dbow+w,d300,n5,w5,mc5,s1e-05,t9>
[('Katy Perry', 0.5954936146736145),
 ('Ariana Grande', 0.5549463033676147),
 ('Thank You, Love (Kana Nishino album)', 0.5462373495101929),
 ('Kōsui (Eito song)', 0.5363353490829468),
 ('Crazy Crazy / Harajuku Iyahoi', 0.5308663249015808),
 ('Big Boys Cry/Beautiful', 0.5303178429603577),
 ('23rd Monster', 0.5298969745635986),
 ('Koi (song)', 0.5275850296020508),
 ('X -Cross-', 0.5255367755889893),
 ('Suzume (song)', 0.525151252746582)]
Doc2Vec<dm/m,d300,n5,w5,mc5,s1e-05,t9>
[('Joanne (album)', 0.5087171196937561),
 ('Artpop', 0.49931594729423523),
 ('The Cure (song)', 0.4835745394229889),
 ('Natali Germanotta', 0.47394222021102905),
 ('Chisato Moritaka', 0.47318926453590393),
 ('Ayumi Hamasaki', 0.47156259417533875),
 ('List of Lady Gaga live performances', 0.4629444479942322),
 ('Joanne Trattoria Cookbook', 0.4615497887134552),
 ('Katy Perry', 0.46151337027549744),
 ('Blackpink', 0.45603010058403015)]


As a result, the DBOW model surfaced artists similar to Lady Gaga in Japan, such as **Ayumi Hamasaki** whose Wiki bio says:

> Ayumi Hamasaki is a Japanese singer, songwriter, record producer, actress, model, spokesperson, and entrepreneur.

So that sounds like a success. It's also the nr. 1 hit in the paper we're replicating.

Similarly, the DM model thought **Kaela Kimura** is the closest hit:

> Kaela Kimura is a Japanese pop rock singer, lyricist, fashion model and television presenter.

Also pretty good.

These results demonstrate that both training modes employed in the original paper are outstanding for calculating similarity between document vectors, word vectors, or a combination of both. The DM mode has the added advantage of being 4x faster to train.

If you wanted to continue working with these trained models, you could save them to disk, to avoid having to re-train the models from scratch every time:

In [19]:
models[0].save('doc2vec_dbow.model')
models[1].save('doc2vec_dm.model')

2022-03-18 19:08:34,623 : INFO : Doc2Vec lifecycle event {'fname_or_handle': 'doc2vec_dbow.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-03-18T19:08:34.622990', 'gensim': '4.1.3.dev0', 'python': '3.8.9 (default, Oct 26 2021, 07:25:53) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-12.2.1-arm64-arm-64bit', 'event': 'saving'}
2022-03-18 19:08:34,641 : INFO : storing np array 'vectors' to doc2vec_dbow.model.dv.vectors.npy
2022-03-18 19:08:40,244 : INFO : storing np array 'vectors' to doc2vec_dbow.model.wv.vectors.npy
2022-03-18 19:08:46,811 : INFO : storing np array 'syn1neg' to doc2vec_dbow.model.syn1neg.npy
2022-03-18 19:08:48,564 : INFO : not storing attribute cum_table
2022-03-18 19:08:56,097 : INFO : saved doc2vec_dbow.model
2022-03-18 19:08:56,098 : INFO : Doc2Vec lifecycle event {'fname_or_handle': 'doc2vec_dm.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-03-18T19:08:56.09876

To continue your doc2vec explorations, refer to the official API documentation in Gensim: https://radimrehurek.com/gensim/models/doc2vec.html